<a href="https://colab.research.google.com/github/jhay20ng/UpData/blob/main/%EC%84%B8_%EA%B0%80%EC%A7%80_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 제 1회 기출동형 모의고사

## (6) 세가지 모델 모두 모델해보고 가장 적합한 알고리즘을 선택하시. 이를 선택한 이유와 모델의 한계점, 보완 가능한 부분을 설명하고, 현업에서 주의할 점 등에 대해 기술하시오.

In [1]:
import os                      # 구글 드라이브에 연결
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd

df = pd.read_csv("/content/gdrive/MyDrive/student_data.csv")
df

,school,sex,paid,activities,famrel,freetime,goout,Dalc,Walc,health,absences,grade,G1,G2
0,GP,F,no,no,4.0,3.0,4.0,1.0,1.0,3.0,6.0,6,5,6
1,GP,F,no,no,5.0,3.0,3.0,1.0,1.0,3.0,4.0,5,5,5
2,GP,F,yes,no,4.0,3.0,2.0,2.0,3.0,3.0,10.0,8,7,8
3,GP,F,yes,yes,3.0,2.0,2.0,1.0,1.0,5.0,2.0,15,15,14
4,GP,F,yes,no,4.0,3.0,2.0,1.0,2.0,5.0,4.0,9,6,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390,MS,M,yes,no,5.0,5.0,4.0,4.0,5.0,4.0,11.0,9,9,9
391,MS,M,no,no,2.0,4.0,5.0,3.0,4.0,2.0,3.0,15,14,16
392,MS,M,no,no,5.0,5.0,3.0,3.0,3.0,3.0,3.0,8,10,8
393,MS,M,no,no,4.0,4.0,1.0,3.0,4.0,5.0,0.0,11,11,12


### 결측치

In [3]:
 # 하나라도 결측치가 있는 내역에 대해 출력
df[df.isna().any(axis=1)]

,school,sex,paid,activities,famrel,freetime,goout,Dalc,Walc,health,absences,grade,G1,G2
9,GP,M,yes,yes,5.0,5.0,NaN,1.0,1.0,5.0,0.0,15,14,15
14,GP,M,no,no,NaN,5.0,2.0,1.0,1.0,3.0,0.0,15,14,16
15,GP,F,no,no,4.0,4.0,4.0,1.0,2.0,2.0,NaN,14,14,14
25,GP,F,yes,no,1.0,2.0,2.0,1.0,NaN,5.0,14.0,8,6,9
39,GP,F,yes,yes,4.0,3.0,1.0,1.0,1.0,NaN,8.0,13,14,13
40,GP,F,no,yes,3.0,3.0,NaN,1.0,2.0,3.0,25.0,9,7,10
49,GP,F,no,yes,4.0,4.0,4.0,1.0,1.0,NaN,2.0,7,7,7
61,GP,F,no,yes,5.0,5.0,5.0,NaN,5.0,5.0,6.0,10,10,8
71,GP,M,no,no,3.0,3.0,3.0,NaN,1.0,3.0,0.0,10,10,10
82,GP,F,yes,no,4.0,4.0,4.0,1.0,1.0,5.0,NaN,6,7,6


In [4]:
# 결측치가 있는 컬럼(독립변수)는 수치형 데이터만 있으므로 KNN방법에 의한 대치 수행​
from sklearn.impute import KNNImputer

# 결측치가 있는 수치형 데이터만 추출
KNN_data = df.drop(columns=['school', 'sex', 'paid', 'activities'])

# 모델링
imputer = KNNImputer()
df_filled = imputer.fit_transform(KNN_data)
df_filled = pd.DataFrame(df_filled, columns=KNN_data.columns)
df[KNN_data.columns] = df_filled

#결측치 확인
df.isna().sum()  #KNN 기법으로 결측치를 모두 대체

school        0
sex           0
paid          0
activities    0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
grade         0
G1            0
G2            0
dtype: int64

### 범주형 변수 인코딩

In [5]:
#회귀 분석을 하기 위해서는 자료의 변환이 필요 (원핫인코딩)
# get_dummies() 함수를 사용하여 이산형 변수에 대한 원핫인코딩을 수행

df = pd.get_dummies(data = df, columns =['school', 'sex', 'paid', 'activities'], drop_first=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   famrel          395 non-null    float64
 1   freetime        395 non-null    float64
 2   goout           395 non-null    float64
 3   Dalc            395 non-null    float64
 4   Walc            395 non-null    float64
 5   health          395 non-null    float64
 6   absences        395 non-null    float64
 7   grade           395 non-null    float64
 8   G1              395 non-null    float64
 9   G2              395 non-null    float64
 10  school_MS       395 non-null    uint8  
 11  sex_M           395 non-null    uint8  
 12  paid_yes        395 non-null    uint8  
 13  activities_yes  395 non-null    uint8  
dtypes: float64(10), uint8(4)
memory usage: 32.5 KB


### 데이터 분할 방법

In [6]:
from sklearn.model_selection import train_test_split

X = df.drop('grade', axis=1)
y = df['grade']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=2022)
print('X Train : ', X_train.shape)
print('X Test : ', X_test.shape)
print('y Train : ', y_train.shape)
print('y Test : ', y_test.shape)


X Train :  (276, 13)
X Test :  (119, 13)
y Train :  (276,)
y Test :  (119,)


### 세 가지 모델링

In [7]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

In [8]:
# 스탠다드 표준
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns )
X_test_scaled = pd.DataFrame(scaler.fit_transform(X_test), columns=X_test.columns )

In [9]:
##SVM

from sklearn.model_selection import GridSearchCV
param_grid = [
    {'C':[0.1, 1, 10, 100], 'gamma':[0.001, 0.01, 0.1, 1, 10]}
]
grid_svm = GridSearchCV(SVR(), param_grid = param_grid, cv = 5)
grid_svm.fit(X_train_scaled, y_train)
result = pd.DataFrame(grid_svm.cv_results_['params'])
result['mean_test_score'] = grid_svm.cv_results_['mean_test_score']
result.sort_values(by='mean_test_score', ascending=False)

,C,gamma,mean_test_score
15,100.0,0.001,0.960941
11,10.0,0.010,0.957198
16,100.0,0.010,0.947652
10,10.0,0.001,0.941800
6,1.0,0.010,0.921494
12,10.0,0.100,0.861286
17,100.0,0.100,0.857767
7,1.0,0.100,0.785883
5,1.0,0.001,0.303419
2,0.1,0.100,0.283644


- C와 gamma는 SVM 모델의 하이퍼파라미터로, 모델의 학습 및 결정 경계의 특성을 제어하는 중요한 매개변수(C는 정규화 파라미터로, 0.1, 1, 10, 100의 후보값을 가짐, gamma는 RBF 커널의 매개변수로, 0.001, 0.01, 0.1, 1, 10의 후보값을 가짐)
- C가 100 이고 gamma가 0.001일때 가장 정확도가 좋음